In [0]:
%run /Workspace/Users/valterlafuentejunior@gmail.com/ProjetoMedalhao/00.config/config

In [0]:
from pyspark.sql.functions import current_date
import pandas as pd
from delta.tables import DeltaTable


In [0]:
tabela_destino = "workspace.bronze_etl.boi_gordo"



In [0]:
df = (
    spark.read
    .option("header", "true") # cabeçalho
    .option("sep", ";")   
    .option("inferSchema", "true") # inferir tipos automaticamente
    .csv("/Volumes/workspace/bronze_etl/boigordo/BoiGordo.csv")
)



In [0]:
df = df.withColumn("data_coleta", current_date())

In [0]:
merge_delta_table(df, tabela_destino)